In [1]:
import os
import tqdm
import re

import pandas as pd
import numpy as np
import string


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from nltk import word_tokenize
from nltk.corpus import stopwords

stop_words=stopwords.words('english')


pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
train.shape,test.shape

((7613, 5), (3263, 4))

In [6]:
print(test.isna().sum())
print(train.isna().sum())

id             0
keyword       26
location    1105
text           0
dtype: int64
id             0
keyword       61
location    2533
text           0
target         0
dtype: int64


In [7]:
def find_hashtags(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"#\w+", tweet)]) or 'no'

def find_mentions(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"@\w+", tweet)]) or 'no'

def find_links(tweet):
    return " ".join([match.group(0)[:] for match in re.finditer(r"https?://\S+", tweet)]) or 'no'

def text_clean(text):
    text = text.lower()
    text = re.sub(r'https?://\S+', '', text) # Remove link
    text = re.sub(r'\n',' ', text) # Remove line breaks
    text = re.sub('[~!@#$\'%&*()-?=]+','',text)
    text = re.sub('\s+', ' ', text).strip() # Remove leading, trailing, and extra spaces
    text = [t for t in text.split() if len(t)>2]
    text= " ".join(text)
    return text

def preprocess_text(df):
    df['hashtags'] = df['text'].apply(lambda x: find_hashtags(x))
    df['mentions'] = df['text'].apply(lambda x: find_mentions(x))
    df['links']    = df['text'].apply(lambda x: find_links(x))
    df['text']     = df['text'].apply(lambda x: text_clean(x))
    return df
train = preprocess_text(train)
test = preprocess_text(test)

train['text_clean'] = train['text'].apply(lambda x: text_clean(x))
test['text_clean'] = test['text'].apply(lambda x: text_clean(x))

In [8]:
def text_stat(df):
    
    df['text_len'] = df['text'].apply(len)
    
    def word_count(x):
        text = [t for t in x.split()]
        t = [a for a in text if a.isalpha()]
        return len(t)
    df['word_count'] = df['text'].apply(lambda x: word_count(x))
    
    df['stop_word_count'] = df['text'].apply(lambda x: len([t for t in str(x).split() if t.lower() not in stop_words]))
    df['punct_count'] = df['text'].apply(lambda x: len([t for t in str(x).split() if t in string.punctuation]))
    df['hashtag_count'] = df['hashtags'].apply(lambda x: len(str(x).split()))

    df['mention_count'] = df['mentions'].apply(lambda x: len(str(x).split()))

    df['link_count'] = df['links'].apply(lambda x: len(str(x).split()))

    df['caps_count'] = df['text'].apply(lambda x: sum(1 for c in str(x) if c.isupper()))
    
    df['caps_ratio'] = df['caps_count'] / df['text_len']
    return df
                                        
                                             
train = text_stat(train)
test = text_stat(test)

In [9]:
train.head()

,id,keyword,location,text,target,hashtags,mentions,links,text_clean,text_len,word_count,stop_word_count,punct_count,hashtag_count,mention_count,link_count,caps_count,caps_ratio
0,1,NaN,NaN,our deeds are the reason this earthquake may a...,1,earthquake,no,no,our deeds are the reason this earthquake may a...,62,11,6,0,1,1,1,0,0.0
1,4,NaN,NaN,forest fire near ronge sask canada,1,no,no,no,forest fire near ronge sask canada,34,6,6,0,1,1,1,0,0.0
2,5,NaN,NaN,all residents asked shelter place are being no...,1,no,no,no,all residents asked shelter place are being no...,112,16,11,0,1,1,1,0,0.0
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1,wildfires,no,no,people receive wildfires evacuation orders cal...,53,6,6,0,1,1,1,0,0.0
4,7,NaN,NaN,just got sent this photo from ruby alaska smok...,1,Alaska wildfires,no,no,just got sent this photo from ruby alaska smok...,80,14,9,0,2,1,1,0,0.0


In [10]:
X = train.text_clean
y=train.target
xtrain, xval, ytrain, yval = train_test_split(X, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)





tfv = TfidfVectorizer(min_df=3,  max_features=3000, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')


ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',max_features=3000,
            ngram_range=(1, 2), stop_words = 'english')


#train and valation for TF-IDF
tfv.fit(list(xtrain) + list(xval))
xtrain_tfv =  tfv.transform(xtrain) 
xval_tfv = tfv.transform(xval)

#train and valation for Count Vectorizer
ctv.fit(list(xtrain) + list(xval))
xtrain_ctv =  ctv.transform(xtrain) 
xval_ctv = ctv.transform(xval)

In [11]:
# Logistic Regression using TF-IDF as input vector
clf = LogisticRegression(C=1)
clf.fit(xtrain_tfv,ytrain)
score = accuracy_score(yval,clf.predict(xval_tfv))
print(round(score,3),end=",")

0.811,

In [12]:
# Logistic Regression using count_vectorizer as input vector
clf = LogisticRegression(C=1)
clf.fit(xtrain_ctv,ytrain)
score = accuracy_score(yval,clf.predict(xval_ctv))
print(round(score,3),end=",")

0.804,

In [13]:
# using sentence vectors as input using glove 50d
embeddings_index = {}
f = open(os.path.expanduser('~/Desktop/glove.6B/glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [14]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(50)
    return v / np.sqrt((v ** 2).sum())

train['embed'] = train['text_clean'].apply(lambda x: sent2vec(x))
train = pd.concat([train,train.embed.apply(pd.Series)],axis=1)

test['embed'] = test['text_clean'].apply(lambda x: sent2vec(x))
test = pd.concat([test,test.embed.apply(pd.Series)],axis=1)


train.head()

,id,keyword,location,text,target,hashtags,mentions,links,text_clean,text_len,word_count,stop_word_count,punct_count,hashtag_count,mention_count,link_count,caps_count,caps_ratio,embed,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,1,NaN,NaN,our deeds are the reason this earthquake may a...,1,earthquake,no,no,our deeds are the reason this earthquake may a...,62,11,6,0,1,1,1,0,0.0,"[0.14650826, -0.0140346205, 0.09169927, -0.115...",0.146508,-0.014035,0.091699,-0.115539,0.158857,-0.013913,0.106585,0.118540,-0.095047,0.092321,-0.065964,0.162121,0.075273,-0.205356,0.143700,0.062220,0.006907,-0.033083,0.014676,0.094826,-0.066985,0.014638,0.106967,-0.131315,0.231296,-0.341661,-0.225792,-0.101255,0.166350,-0.150639,0.573699,0.071956,-0.087154,-0.082364,-0.068041,-0.004233,0.098029,-0.226833,0.046584,-0.042759,-0.108152,-0.046740,-0.022183,0.021913,-0.047678,0.080133,-0.121086,0.040053,0.037031,-0.006494
1,4,NaN,NaN,forest fire near ronge sask canada,1,no,no,no,forest fire near ronge sask canada,34,6,6,0,1,1,1,0,0.0,"[-0.0017933911, 0.029155271, -0.054189917, 0.1...",-0.001793,0.029155,-0.054190,0.117770,-0.192713,-0.127842,-0.134595,0.064742,0.138415,-0.112464,0.080753,-0.149033,0.134145,-0.171783,-0.220011,0.039205,0.116196,0.153646,-0.276183,-0.042536,-0.064249,0.014163,-0.051649,0.043591,-0.011546,-0.195259,0.045397,0.143953,0.059216,0.011864,0.588915,-0.082076,-0.063108,-0.105032,0.021259,-0.039399,-0.049555,-0.174797,0.151960,0.109565,-0.051535,-0.027562,0.061412,-0.164586,-0.066661,0.166252,-0.067376,-0.163471,0.090031,-0.124005
2,5,NaN,NaN,all residents asked shelter place are being no...,1,no,no,no,all residents asked shelter place are being no...,112,16,11,0,1,1,1,0,0.0,"[0.20231037, -0.05466711, 0.061222546, -0.1012...",0.202310,-0.054667,0.061223,-0.101279,0.091641,-0.147279,-0.169476,0.142209,0.043089,-0.199310,-0.029161,-0.099416,0.098060,-0.081225,0.126205,0.057555,-0.034376,-0.071708,-0.035077,-0.116812,0.075351,0.071784,-0.036543,-0.007304,-0.052998,-0.319805,0.037618,-0.031714,0.054229,-0.036597,0.713225,0.062801,-0.135522,-0.139868,0.040279,0.073573,0.150981,-0.087271,0.048833,0.016525,-0.084066,0.010165,0.108419,-0.008033,0.038038,-0.032123,-0.133263,-0.004550,0.012289,-0.001830
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1,wildfires,no,no,people receive wildfires evacuation orders cal...,53,6,6,0,1,1,1,0,0.0,"[0.1878083, -0.08654569, 0.11575288, -0.089645...",0.187808,-0.086546,0.115753,-0.089645,-0.093891,-0.045978,-0.142584,0.076029,0.111623,-0.050814,-0.046718,-0.169389,0.219136,-0.122975,0.139266,-0.061540,-0.084932,-0.007627,-0.081784,-0.140387,-0.006593,-0.000532,-0.028439,-0.060760,-0.011429,-0.288055,-0.024023,-0.098726,0.064222,0.013528,0.685800,0.058760,-0.137248,-0.147099,0.005158,0.029386,0.061265,-0.155796,0.076608,0.021138,-0.152750,0.056577,0.177702,0.005594,0.009732,0.056476,-0.119409,0.022680,-0.069896,0.001809
4,7,NaN,NaN,just got sent this photo from ruby alaska smok...,1,Alaska wildfires,no,no,just got sent this photo from ruby alaska smok...,80,14,9,0,2,1,1,0,0.0,"[-0.02485075, 0.081003025, 0.07718478, -0.0629...",-0.024851,0.081003,0.077185,-0.062967,-0.101143,-0.022318,-0.243590,-0.004097,0.097521,-0.058729,-0.105047,-0.080770,0.096663,-0.021835,0.089284,-0.020524,-0.103236,0.066144,-0.199384,-0.147853,0.016370,0.128381,0.099011,-0.064244,0.049011,-0.423425,-0.027240,0.174811,0.112701,-0.149042,0.578898,0.007009,-0.088297,-0.163335,-0.013597,0.115901,-0.076797,-0.093262,0.199200,-0.029054,-0.010735,0.142692,0.064742,-0.053606,0.065245,0.082839,0.052499,-0.137691,-0.047772,-0.088368


In [15]:
cols = ['id','keyword','location','text','target','hashtags','mentions','links','text_clean','embed',\
       'text_len','punct_count','hashtag_count','mention_count','link_count','caps_ratio']

X = train[[col for col in train.columns if col not in cols]]
y=train.target

xtrain, xval, ytrain, yval = train_test_split(X, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

#test_ = test[X.columns]
clf = LogisticRegression(C=1)
clf.fit(xtrain,ytrain)
score = accuracy_score(yval,clf.predict(xval))
print(round(score,3),end=",")

0.806,

In [46]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [48]:
from keras import *
from keras.layers import Dense,Dropout,BatchNormalization,Activation
from keras.callbacks import EarlyStopping,ModelCheckpoint

X = train[list(np.arange(0,50,1))]
y = train.target


def keras_model(): 
    model = Sequential()

    model.add(Dense(64, input_shape=[len(normed_train_data.columns)], activation='relu'))
    model.add(Dense(128,activation='relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(256,activation='relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(1,activation='sigmoid')) 

    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=[f1_m])
    return model

model = keras_model()
history = model.fit(X, y,validation_split=0.1,
                    epochs=500, 
                    batch_size=100,
                    verbose=1)

Train on 6851 samples, validate on 762 samples
Epoch 1/500
6851/6851 [==============================] - 1s 86us/step - loss: 0.5568 - f1_m: 0.6130 - val_loss: 0.4604 - val_f1_m: 0.7192
Epoch 2/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.4737 - f1_m: 0.7256 - val_loss: 0.4433 - val_f1_m: 0.7096
Epoch 3/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.4593 - f1_m: 0.7410 - val_loss: 0.4373 - val_f1_m: 0.7176
Epoch 4/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.4499 - f1_m: 0.7425 - val_loss: 0.4278 - val_f1_m: 0.7112
Epoch 5/500
6851/6851 [==============================] - 0s 16us/step - loss: 0.4417 - f1_m: 0.7469 - val_loss: 0.4298 - val_f1_m: 0.7277
Epoch 6/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.4325 - f1_m: 0.7526 - val_loss: 0.4242 - val_f1_m: 0.7274
Epoch 7/500
6851/6851 [==============================] - 0s 16us/step - loss: 0.4274 - f1_m: 0.7563 - val_loss: 0.4280 - val_

Epoch 118/500
6851/6851 [==============================] - 0s 18us/step - loss: 0.0996 - f1_m: 0.9516 - val_loss: 1.4171 - val_f1_m: 0.6955
Epoch 119/500
6851/6851 [==============================] - 0s 18us/step - loss: 0.0989 - f1_m: 0.9520 - val_loss: 1.5171 - val_f1_m: 0.6757
Epoch 120/500
6851/6851 [==============================] - 0s 25us/step - loss: 0.0988 - f1_m: 0.9497 - val_loss: 1.4722 - val_f1_m: 0.6926
Epoch 121/500
6851/6851 [==============================] - 0s 28us/step - loss: 0.0978 - f1_m: 0.9529 - val_loss: 1.4732 - val_f1_m: 0.6992
Epoch 122/500
6851/6851 [==============================] - 0s 27us/step - loss: 0.0989 - f1_m: 0.9505 - val_loss: 1.5311 - val_f1_m: 0.6895
Epoch 123/500
6851/6851 [==============================] - 0s 20us/step - loss: 0.0948 - f1_m: 0.9529 - val_loss: 1.6399 - val_f1_m: 0.6626
Epoch 124/500
6851/6851 [==============================] - 0s 22us/step - loss: 0.1008 - f1_m: 0.9502 - val_loss: 1.5351 - val_f1_m: 0.6872
Epoch 125/500
6851/6

6851/6851 [==============================] - 0s 17us/step - loss: 0.0751 - f1_m: 0.9634 - val_loss: 2.1540 - val_f1_m: 0.6991
Epoch 235/500
6851/6851 [==============================] - 0s 15us/step - loss: 0.0606 - f1_m: 0.9670 - val_loss: 2.4749 - val_f1_m: 0.7090
Epoch 236/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.0739 - f1_m: 0.9627 - val_loss: 2.3220 - val_f1_m: 0.7102
Epoch 237/500
6851/6851 [==============================] - 0s 18us/step - loss: 0.0830 - f1_m: 0.9580 - val_loss: 2.1177 - val_f1_m: 0.6957
Epoch 238/500
6851/6851 [==============================] - 0s 18us/step - loss: 0.0719 - f1_m: 0.9616 - val_loss: 2.3170 - val_f1_m: 0.6828
Epoch 239/500
6851/6851 [==============================] - 0s 18us/step - loss: 0.0667 - f1_m: 0.9646 - val_loss: 2.2285 - val_f1_m: 0.6947
Epoch 240/500
6851/6851 [==============================] - 0s 22us/step - loss: 0.0637 - f1_m: 0.9670 - val_loss: 2.3746 - val_f1_m: 0.6970
Epoch 241/500
6851/6851 [=========

6851/6851 [==============================] - 0s 17us/step - loss: 0.0600 - f1_m: 0.9670 - val_loss: 2.5176 - val_f1_m: 0.7134
Epoch 351/500
6851/6851 [==============================] - 0s 18us/step - loss: 0.0544 - f1_m: 0.9691 - val_loss: 2.3955 - val_f1_m: 0.7177
Epoch 352/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.0568 - f1_m: 0.9697 - val_loss: 2.5015 - val_f1_m: 0.7098
Epoch 353/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.0586 - f1_m: 0.9681 - val_loss: 2.4413 - val_f1_m: 0.7011
Epoch 354/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.0545 - f1_m: 0.9691 - val_loss: 2.5645 - val_f1_m: 0.6992
Epoch 355/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.0587 - f1_m: 0.9709 - val_loss: 2.6986 - val_f1_m: 0.6989
Epoch 356/500
6851/6851 [==============================] - 0s 17us/step - loss: 0.0546 - f1_m: 0.9690 - val_loss: 2.4186 - val_f1_m: 0.6886
Epoch 357/500
6851/6851 [=========

6851/6851 [==============================] - 0s 17us/step - loss: 0.0526 - f1_m: 0.9711 - val_loss: 2.5517 - val_f1_m: 0.7062
Epoch 467/500
6851/6851 [==============================] - 0s 15us/step - loss: 0.0521 - f1_m: 0.9722 - val_loss: 2.8389 - val_f1_m: 0.6898
Epoch 468/500
6851/6851 [==============================] - 0s 15us/step - loss: 0.0559 - f1_m: 0.9690 - val_loss: 2.9393 - val_f1_m: 0.7064
Epoch 469/500
6851/6851 [==============================] - 0s 16us/step - loss: 0.0510 - f1_m: 0.9710 - val_loss: 2.8918 - val_f1_m: 0.6580
Epoch 470/500
6851/6851 [==============================] - 0s 15us/step - loss: 0.0509 - f1_m: 0.9708 - val_loss: 2.8521 - val_f1_m: 0.6933
Epoch 471/500
6851/6851 [==============================] - 0s 15us/step - loss: 0.0497 - f1_m: 0.9709 - val_loss: 2.8136 - val_f1_m: 0.6861
Epoch 472/500
6851/6851 [==============================] - 0s 15us/step - loss: 0.0519 - f1_m: 0.9693 - val_loss: 2.8372 - val_f1_m: 0.7356
Epoch 473/500
6851/6851 [=========

In [49]:
pred = model.predict_classes(test[X.columns])

In [52]:
temp = test.copy()
temp['target'] = pred
temp.index=temp.id
temp = temp[['target']]
temp.to_csv('try4.csv')
temp.head()

,target
id,
0,0
2,1
3,1
9,1
11,1
